## Extraction de données depuis Wikidata

Ce notebook illustre la méthode utilisée pour extraire des données depuis **Wikidata** à l’aide de requêtes **SPARQL**.

Les informations récupérées concernent les **entraîneurs ayant officié en Serie A**, leurs **clubs entraînés**, leurs **dates de mandat**, ainsi que leur **parcours de joueur**.

La majorité des QIDs a été obtenue en croisant :
- la **catégorie Wikipédia** `Category:Serie_A_managers`  
- avec les correspondances sur Wikidata à l’aide de requêtes SPARQL.

Les requêtes ont été exécutées avec le module **SPARQLWrapper** en Python, avec une logique de **pause entre chaque appel** (pour éviter les erreurs 429) et une gestion des erreurs automatisée.


## Récupération des QIDs des entraîneurs depuis Wikipedia

Pour obtenir la liste complète des entraîneurs de Serie A présents sur la page Wikipedia suivante :

[Category:Serie A managers](https://en.wikipedia.org/wiki/Category:Serie_A_managers)

j'ai utilisé l'outil **[PetScan](https://petscan.wmflabs.org/)**, qui permet d'extraire automatiquement les identifiants Wikidata (QIDs) associés à chaque page de catégorie Wikipedia.  
Cela m'a permis de créer un fichier `SerieA_managers.csv` contenant une ligne par entraîneur, avec le **titre de la page Wikipedia** et son **identifiant Wikidata**.


## Exemple de requête type utilisée pour extraire les données de Wikidata

Ci-dessous un exemple représentatif du type de requêtes SPARQL utilisées pour construire notre base de données.

La requête suivante, exécutée en Python à l’aide de la librairie `SPARQLWrapper`, permet de récupérer, pour chaque entraîneur identifié par un QID :
- les **équipes** qu’il a entraînées (`entityLabel`),
- les **QIDs** de ces équipes (`entity_qid`),
- les **dates de début et de fin** de ses mandats (`startDate`, `endDate`),
- uniquement si ces informations sont disponibles sur **Wikidata** via la propriété `P6087` (*coach of sports team*).

Cette logique a été appliquée pour créer les autres fichiers csv.

Un délai (`time.sleep(2)`) et un mécanisme de relance automatique en cas d’erreur (`retry`) ont été intégrés pour éviter le blocage de l’API SPARQL de Wikidata.


In [ ]:
import pandas as pd
import time
from SPARQLWrapper import SPARQLWrapper, JSON

# Charger le fichier
df_qids = pd.read_csv("SerieA_managers.csv")
qids = df_qids["Wikidata"].dropna().unique()

# Config SPARQL
sparql = SPARQLWrapper("https://query.wikidata.org/sparql")
sparql.setReturnFormat(JSON)
sparql.addCustomHttpHeader("User-Agent", "noto-student-wikidata/1.0 (mailto:ton.email@epfl.ch)")

# Fonction de requête avec retry
def get_mandates_with_retry(qid, max_tries=5):
    tries = 0
    while tries < max_tries:
        try:
            query = f"""
            SELECT ?entity ?entityLabel ?startDate ?endDate WHERE {{
              wd:{qid} p:P6087 ?statement.
              ?statement ?prop ?entity.
              OPTIONAL {{ ?statement pq:P580 ?startDate. }}
              OPTIONAL {{ ?statement pq:P582 ?endDate. }}
              FILTER(STRSTARTS(STR(?prop), "http://www.wikidata.org/prop/"))
              FILTER(STRSTARTS(STR(?entity), "http://www.wikidata.org/entity/Q"))
              SERVICE wikibase:label {{ bd:serviceParam wikibase:language "en". }}
            }}
            ORDER BY ?startDate
            """
            sparql.setQuery(query)
            results = sparql.query().convert()
            return results["results"]["bindings"]
        except Exception as e:
            print(f" Erreur tentative {tries+1}/5 pour {qid} : {e}")
            tries += 1
            time.sleep(10)
    return None

# Boucle principale
all_data = []
errors = []

for qid in qids:
    print(f"🔎 Traitement de {qid}...")
    results = get_mandates_with_retry(qid)
    if results is None:
        errors.append(qid)
        continue
    for res in results:
        all_data.append({
            "coach_qid": qid,
            "entity_qid": res["entity"]["value"].split("/")[-1],
            "entityLabel": res["entityLabel"]["value"],
            "startDate": res.get("startDate", {}).get("value", ""),
            "endDate": res.get("endDate", {}).get("value", "")
        })
    time.sleep(2)

# Sauvegarde des données
df_out = pd.DataFrame(all_data)
df_out.to_csv("mandats_entraineurs2.csv", index=False)
print("✅ Export terminé → mandats_entraineurs2.csv")

# QID échoués
if errors:
    pd.DataFrame(errors, columns=["qid"]).to_csv("qids_non_traite.csv", index=False)
    print(f" {len(errors)} QID non traités. Sauvegardés dans qids_non_traite.csv.")